<a href="https://colab.research.google.com/github/BIwithRuben/vulnerabilidad_climatica_mex/blob/Marcio/TEC_UCM_Notebook_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marcio's notebook for the project ENIGH TEC-UCM

Files are too large, some of them with more than 500Mb.  
I used python's library POLARS, and confirmed if it is more efficient than pandas in terms of memory usage and performnace.

### CONSIDERANDO ARCHIVOS EN MI ORDENADORA (sin API)

#### IMPORTING LIBRARIES

In [1]:
import polars as pl

#### CODE

In [2]:
# Reading files with POLARS

file_name = 'hogares.csv'  # concentradohogar, erogaciones, gastoshogar, hogares, ingresos, viviendas

df = pl.read_csv(
    file_name,
    infer_schema_length=10000,
    schema_overrides={"tipo_viv": pl.Utf8},
    #ignore_errors=True,
    null_values="&"
)

print(df.shape)
print(df.head())
print(df.schema)
print(f'Columns {df.columns}')

(86541, 148)
shape: (5, 148)
┌───────────┬──────────┬───────────┬────────────┬───┬─────────┬─────────┬─────┬────────┐
│ folioviv  ┆ foliohog ┆ huespedes ┆ huesp_come ┆ … ┆ entidad ┆ est_dis ┆ upm ┆ factor │
│ ---       ┆ ---      ┆ ---       ┆ ---        ┆   ┆ ---     ┆ ---     ┆ --- ┆ ---    │
│ i64       ┆ i64      ┆ i64       ┆ str        ┆   ┆ i64     ┆ i64     ┆ i64 ┆ i64    │
╞═══════════╪══════════╪═══════════╪════════════╪═══╪═════════╪═════════╪═════╪════════╡
│ 100001901 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 1       ┆ 1   ┆ 207    │
│ 100001902 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 1       ┆ 1   ┆ 207    │
│ 100001904 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 1       ┆ 1   ┆ 207    │
│ 100001905 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 1       ┆ 1   ┆ 207    │
│ 100002501 ┆ 1        ┆ 0         ┆            ┆ … ┆ 1       ┆ 2       ┆ 2   ┆ 196    │
└───────────┴──────────┴───────────┴────────────┴───┴─────────┴─────────┴─────┴──

# 📊 Variables by Relevant Information

## CLAVE PRIMARIA (Primary Key)
| Variable | Definition |
| :--- | :--- |
| **folioviv** | Identificador de la vivienda compuesto por dos dígitos con la clave de la entidad federativa, uno con el ámbito (urbano, código diferente a 6; rural, código 6), cuatro dígitos del número consecutivo de la UPM, un dígito de la decena de levantamiento y dos dígitos con un número consecutivo para la vivienda seleccionada. |

---

## Seguridad alimentaria y nutricional (LEVÉ, MODERADA Y GRAVE) (Food and Nutritional Security (MILD, MODERATE, AND SEVERE))

- **Sin comida (acc_alim2)**
  - Type: **BOOLEAN**
  - Definition: En los últimos tres meses, por falta de dinero o recursos ¿alguna vez se quedaron sin comida? (In the last three months, due to lack of money or resources, did you ever run out of food?)
- **Sintió hambre y no comió (acc_alim7)**
  - Type: **BOOLEAN**
  - Definition: En los últimos tres meses, por falta de dinero o recursos, ¿alguna vez usted o alguna persona adulta en su hogar sintió hambre pero no comió? (In the last three months, due to lack of money or resources, did you or any adult in your household ever feel hungry but not eat?)
- **Mendigar por comida (acc_alim9)**
  - Type: **BOOLEAN**
  - Definition: En los últimos tres meses, por falta de dinero o recursos ¿alguna vez tuvieron que hacer algo que hubieran preferido no hacer para conseguir comida, tal como mendigar (pedir limosna) o mandar a los niños a trabajar? (In the last three months, due to lack of money or resources, did you ever have to do something you would have preferred not to do to get food, such as begging or sending children to work?)

---

## Vulnerabilidad económica / Inclusión digital (Economic Vulnerability / Digital Inclusion)

- **Teléfono celular (celular)**
  - Type: **BOOLEAN**
  - Definition: ¿Este hogar cuenta con teléfono móvil o celular? (Does this household have a mobile or cell phone?)

---

## Accesso a los servicios de salud (Access to Health Services)

- **Horas en llegar al hospital (tsalud1_h)**
  - Type: **NUMERICO CONTINUO (CONTINUOUS NUMERIC)**
  - Definition: En caso de emergencia y considerando el medio de transporte que utilizaría, ¿cuánto tiempo cree que tardaría en llegar a un hospital en donde lo(a) pudieran atender? (In case of emergency and considering the means of transport you would use, how long do you think it would take to get to a hospital where you could be treated?)
- **Minutos en llegar al hospital (tsalud1_m)**
  - Type: **NUMERICO CONTINUO (CONTINUOUS NUMERIC)**
  - Definition: En caso de emergencia y considerando el medio de transporte que utilizaría, ¿cuánto tiempo cree que tardaría en llegar a un hospital en donde lo(a) pudieran atender? (Same definition as above, likely capturing minutes for precision).

---

## Vulnerabilidad climatica (Climatic Vulnerability)

- **Afectación por cambio climático (camb_clim)**
  - Type: **BOOLEAN**
  - Definition: De 2023 a la fecha, ¿en este hogar se vieron afectados por algún fenómeno climático como sequía, inundación, helada, incendio, huracán o tormenta tropical? (From 2023 to date, was this household affected by any climatic phenomenon such as drought, flood, frost, fire, hurricane, or tropical storm?)
- **Afectación por sequía (f_sequia)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha? (What was the last climatic phenomenon that affected them from 2023 to date?)
- **Afectación por inundación (f_inunda)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por helada (f_helada)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por incendio (f_incendio)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por huracán o ciclón (f_huracan)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por deslizamiento de tierras (f_desliza)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Afectación por otro fenómeno climático (f_otro)**
  - Type: **BOOLEAN**
  - Definition: ¿Cuál fue el último fenómeno climático que los afectó de 2023 a la fecha?
- **Daños a la vivienda por fenómeno climático (af_viv)**
  - Type: **BOOLEAN**
  - Definition: Respecto al fenómeno climático referido, ¿cómo les afectó? (Regarding the referred climatic phenomenon, how did it affect them?)
- **Perdida de empleo por fenómeno climático (af_empleo)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Daño en su negocio por fenómeno climático (af_negocio)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Afectación a cultivos por fenómeno climático (af_cultivo)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Perdida de animales de trabajo por fenómeno climático (af_trabajo)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Afectaciones graves a su salud por fenómeno climático (af_salud)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)
- **Otro tipo de afectación por fenómeno climático (af_otro)**
  - Type: **BOOLEAN**
  - Definition: (Implied: How did the climatic phenomenon affect them?)

---

In [7]:
# Select specific columns

# FROM viviendas.csv
#selected = df.select(['folioviv', 'tot_resid', 'mat_pared', 'mat_techos', 'ab_agua', 'agua_noe', 'dotac_agua', 'sanit_agua', 'drenaje'])

# FROM concentradohogar.csv
#selected = df.select(['folioviv', 'foliohog', 'ing_cor', 'gasto_mon', 'ubica_geo', 'tam_loc'])

# FROM erogaciones.csv
#selected = df.select(['folioviv', 'foliohog', 'ing_cor', 'gasto_mon', 'ubica_geo', 'tam_loc'])

# FROM hogares.csv
selected = df.select(['folioviv',
                      'acc_alim2',
                      'acc_alim7',
                      'acc_alim9',
                      'celular',
                      'tsalud1_h',
                      'tsalud1_m',
                      'camb_clim',
                      'f_sequia',
                      'f_inunda',
                      'f_helada',
                      'f_incendio',
                      'f_huracan',
                      'f_desliza',
                      'f_otro',
                      'af_viv',
                      'af_empleo',
                      'af_cultivo',
                      'af_trabajo',
                      'af_salud',
                      'af_otro'
                    ]
                  )


In [8]:
# Storing in a dictionary with dynamic key
dfs = {}
dfs[f"df_{file_name.split('.')[0]}"] = selected

In [9]:
# Printing the New_Dataframe new_df
print(f'Selected columns to df_{file_name.split('.')[0]}.csv from {file_name}')
dfs[f"df_{file_name.split('.')[0]}"]  # df_concentradohogar, df_erogaciones, df_gastoshogar, df_hogares, df_ingresos, df_viviendas

Selected columns to df_hogares.csv from hogares.csv


folioviv,acc_alim2,acc_alim7,acc_alim9,celular,tsalud1_h,tsalud1_m,camb_clim,f_sequia,f_inunda,f_helada,f_incendio,f_huracan,f_desliza,f_otro,af_viv,af_empleo,af_cultivo,af_trabajo,af_salud,af_otro
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
100001901,2,null,null,1,0,5,2,null,null,null,null,null,null,null,null,null,null,null,null,null
100001902,2,null,null,1,1,0,2,null,null,null,null,null,null,null,null,null,null,null,null,null
100001904,2,1,2,1,0,5,2,null,null,null,null,null,null,null,null,null,null,null,null,null
100001905,2,2,2,1,0,5,2,null,null,null,null,null,null,null,null,null,null,null,null,null
100002501,2,null,null,1,0,5,2,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3100091903,2,null,null,1,0,10,2,null,null,null,null,null,null,null,null,null,null,null,null,null
3100091904,2,null,null,1,0,10,2,null,null,null,null,null,null,null,null,null,null,null,null,null
3100091905,2,null,null,1,1,0,2,null,null,null,null,null,null,null,null,null,null,null,null,null


In [10]:
# Build dynamic name for output file
output_file = f"selected_{file_name}"

# Save the DataFrame to CSV
dfs[f"df_{file_name.split('.')[0]}"].write_csv(output_file)

print(f"File saved: {output_file}")


File saved: selected_hogares.csv
